In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import sys
import os
import zipfile
import json
from tqdm import tqdm
import ast
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from PIL import Image, ImageDraw
import io
from IPython.display import display
import matplotlib.pyplot as plt

# 시설 질병

In [4]:
ROOT_DIR = 'D:/rawdata_disease/house/train'

### 라벨링데이터

In [3]:
label = os.listdir(ROOT_DIR)
label = [i for i in label if 'label' in i]
label

['label_tomato_0.normal.zip',
 'label_tomato_1.disease.zip',
 'label_tomato_9.disease.zip']

#### 정상

In [7]:
with zipfile.ZipFile(os.path.join(ROOT_DIR, label[0]), 'r') as target_file:
    file_list = target_file.namelist()

In [8]:
file_list

['V006_77_0_00_11_01_13_0_c01_20201013_0000_S01.jpg.json',
 'V006_77_0_00_11_01_13_0_c01_20201013_0000_S01_1.jpg.json',
 'V006_77_0_00_11_01_13_0_c01_20201013_0001_S01.jpg.json',
 'V006_77_0_00_11_01_13_0_c01_20201013_0001_S01_1.jpg.json',
 'V006_77_0_00_11_01_13_0_c01_20201013_0002_S01.jpg.json',
 'V006_77_0_00_11_01_13_0_c01_20201013_0002_S01_1.jpg.json',
 'V006_77_0_00_11_01_13_0_c01_20201013_0003_S01.jpg.json',
 'V006_77_0_00_11_01_13_0_c01_20201013_0003_S01_1.jpg.json',
 'V006_77_0_00_11_01_13_0_c01_20201013_0004_S01.jpg.json',
 'V006_77_0_00_11_01_13_0_c01_20201013_0004_S01_1.jpg.json',
 'V006_77_0_00_11_01_13_0_c01_20201013_0005_S01.jpg.json',
 'V006_77_0_00_11_01_13_0_c01_20201013_0005_S01_1.jpg.json',
 'V006_77_0_00_11_01_13_0_c01_20201013_0006_S01.jpg.json',
 'V006_77_0_00_11_01_13_0_c01_20201013_0006_S01_1.jpg.json',
 'V006_77_0_00_11_01_13_0_c01_20201013_0007_S01.jpg.json',
 'V006_77_0_00_11_01_13_0_c01_20201013_0007_S01_1.jpg.json',
 'V006_77_0_00_11_01_13_0_c01_20201013_0

In [9]:
len(file_list)

13509

In [10]:
file_list[0]

'V006_77_0_00_11_01_13_0_c01_20201013_0000_S01.jpg.json'

In [30]:
normal_json_data = []

for i in tqdm(range(len(file_list))):
    with zipfile.ZipFile(os.path.join(ROOT_DIR, label[0]), 'r') as zip_ref:
        with zip_ref.open(file_list[i]) as file:
            normal_json_data.append(json.load(file))

normal_json_data

100%|████████████████████████████████████████████████████████████████████████████| 13509/13509 [28:56<00:00,  7.78it/s]


[{'description': {'image': 'V006_77_0_00_11_01_13_0_c01_20201013_0000_S01.jpg',
   'date': '2020/10/13',
   'worker': '',
   'height': 3024,
   'width': 4032,
   'task': 77,
   'type': 0,
   'region': None},
  'annotations': {'disease': 0,
   'crop': 11,
   'area': 1,
   'grow': 13,
   'risk': 0,
   'points': [{'xtl': 1864, 'ytl': 1516, 'xbr': 2292, 'ybr': 1880}]}},
 {'description': {'image': 'V006_77_0_00_11_01_13_0_c01_20201013_0000_S01_1.jpg',
   'date': '2020/10/13',
   'worker': '',
   'height': 3024,
   'width': 4032,
   'task': 77,
   'type': 0,
   'region': None},
  'annotations': {'disease': 0,
   'crop': 11,
   'area': 1,
   'grow': 13,
   'risk': 0,
   'points': [{'xtl': 1722, 'ytl': 436, 'xbr': 2167, 'ybr': 676}]}},
 {'description': {'image': 'V006_77_0_00_11_01_13_0_c01_20201013_0001_S01.jpg',
   'date': '2020/10/13',
   'worker': '',
   'height': 3024,
   'width': 4032,
   'task': 77,
   'type': 0,
   'region': None},
  'annotations': {'disease': 0,
   'crop': 11,
   'are

In [32]:
normal_df_list = []
for i in tqdm(range(len(normal_json_data))):
    data = {
    'image': normal_json_data[i]['description']['image'],
    'date': normal_json_data[i]['description']['date'],
    'worker': normal_json_data[i]['description']['worker'],
    'height': normal_json_data[i]['description']['height'],
    'width': normal_json_data[i]['description']['width'],
    'task': normal_json_data[i]['description']['task'],
    'type': normal_json_data[i]['description']['type'],
    'region': normal_json_data[i]['description']['region'],
    'disease' : normal_json_data[i]['annotations']['disease'],
    'crop' : normal_json_data[i]['annotations']['crop'],
    'area' : normal_json_data[i]['annotations']['area'],
    'grow' : normal_json_data[i]['annotations']['grow'],
    'risk' : normal_json_data[i]['annotations']['risk'],
    'points' : normal_json_data[i]['annotations']['points']
    }
    normal_df_list.append(data)
    
normal_df = pd.DataFrame(normal_df_list)
normal_df

100%|████████████████████████████████████████████████████████████████████████| 13509/13509 [00:00<00:00, 199078.94it/s]


,image,date,worker,height,width,task,type,region,disease,crop,area,grow,risk,points
0,V006_77_0_00_11_01_13_0_c01_20201013_0000_S01.jpg,2020/10/13,,3024,4032,77,0,None,0,11,1,13,0,"[{'xtl': 1864, 'ytl': 1516, 'xbr': 2292, 'ybr'..."
1,V006_77_0_00_11_01_13_0_c01_20201013_0000_S01_...,2020/10/13,,3024,4032,77,0,None,0,11,1,13,0,"[{'xtl': 1722, 'ytl': 436, 'xbr': 2167, 'ybr':..."
2,V006_77_0_00_11_01_13_0_c01_20201013_0001_S01.jpg,2020/10/13,,3024,4032,77,0,None,0,11,1,13,0,"[{'xtl': 2150, 'ytl': 100, 'xbr': 2727, 'ybr':..."
3,V006_77_0_00_11_01_13_0_c01_20201013_0001_S01_...,2020/10/13,,3024,4032,77,0,None,0,11,1,13,0,"[{'xtl': 2147, 'ytl': 102, 'xbr': 2729, 'ybr':..."
4,V006_77_0_00_11_01_13_0_c01_20201013_0002_S01.jpg,2020/10/13,,3024,4032,77,0,None,0,11,1,13,0,"[{'xtl': 1979, 'ytl': 832, 'xbr': 2719, 'ybr':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13504,V006_77_0_00_11_05_13_0_c42_20201028_0066_S01_...,2020/10/28,,3024,4032,77,0,None,0,11,5,13,0,"[{'xtl': 3, 'ytl': 737, 'xbr': 4027, 'ybr': 20..."
13505,V006_77_0_00_11_05_13_0_c42_20201028_0067_S01.jpg,2020/10/28,,3024,4032,77,0,None,0,11,5,13,0,"[{'xtl': 0, 'ytl': 905, 'xbr': 4023, 'ybr': 17..."
13506,V006_77_0_00_11_05_13_0_c42_20201028_0067_S01_...,2020/10/28,,3024,4032,77,0,None,0,11,5,13,0,"[{'xtl': 0, 'ytl': 953, 'xbr': 4031, 'ybr': 16..."
13507,V006_77_0_00_11_05_13_0_c42_20201028_0068_S01.jpg,2020/10/28,,3024,4032,77,0,None,0,11,5,13,0,"[{'xtl': 0, 'ytl': 1105, 'xbr': 4032, 'ybr': 2..."


In [33]:
normal_df.to_csv('normal_df.csv')

#### 질병1

In [19]:
with zipfile.ZipFile(os.path.join(ROOT_DIR, label[1]), 'r') as target_file:
    file_list = target_file.namelist()

In [20]:
disease1_json_data = []

for i in tqdm(range(len(file_list))):
    with zipfile.ZipFile(os.path.join(ROOT_DIR, label[1]), 'r') as zip_ref:
        with zip_ref.open(file_list[i]) as file:
            disease1_json_data.append(json.load(file))

disease1_json_data

100%|██████████████████████████████████████████████████████████████████████████████| 1967/1967 [00:31<00:00, 63.02it/s]


[{'description': {'image': 'V006_77_1_18_11_03_13_1_3248b_20201005_57.jpg',
   'date': '2020/10/05',
   'worker': '',
   'height': 3024,
   'width': 4032,
   'task': 77,
   'type': 1,
   'region': 4},
  'annotations': {'disease': 18,
   'crop': 11,
   'area': 3,
   'grow': 13,
   'risk': 1,
   'points': [{'xtl': 908, 'ytl': 916, 'xbr': 3104, 'ybr': 2312}]}},
 {'description': {'image': 'V006_77_1_18_11_03_13_1_3248b_20201012_60.jpg',
   'date': '2020/10/12',
   'worker': '',
   'height': 3024,
   'width': 4032,
   'task': 77,
   'type': 1,
   'region': 4},
  'annotations': {'disease': 18,
   'crop': 11,
   'area': 3,
   'grow': 13,
   'risk': 1,
   'points': [{'xtl': 1108, 'ytl': 556, 'xbr': 3356, 'ybr': 2260}]}},
 {'description': {'image': 'V006_77_1_18_11_03_13_1_3248b_20201012_61.jpg',
   'date': '2020/10/12',
   'worker': '',
   'height': 3024,
   'width': 4032,
   'task': 77,
   'type': 1,
   'region': 4},
  'annotations': {'disease': 18,
   'crop': 11,
   'area': 3,
   'grow': 13,

In [21]:
disease1_df_list = []
for i in tqdm(range(len(disease1_json_data))):
    data = {
    'image': disease1_json_data[i]['description']['image'],
    'date': disease1_json_data[i]['description']['date'],
    'worker': disease1_json_data[i]['description']['worker'],
    'height': disease1_json_data[i]['description']['height'],
    'width': disease1_json_data[i]['description']['width'],
    'task': disease1_json_data[i]['description']['task'],
    'type': disease1_json_data[i]['description']['type'],
    'region': disease1_json_data[i]['description']['region'],
    'disease' : disease1_json_data[i]['annotations']['disease'],
    'crop' : disease1_json_data[i]['annotations']['crop'],
    'area' : disease1_json_data[i]['annotations']['area'],
    'grow' : disease1_json_data[i]['annotations']['grow'],
    'risk' : disease1_json_data[i]['annotations']['risk'],
    'points' : disease1_json_data[i]['annotations']['points']
    }
    disease1_df_list.append(data)
    
disease1_df = pd.DataFrame(disease1_df_list)
disease1_df

100%|██████████████████████████████████████████████████████████████████████████| 1967/1967 [00:00<00:00, 219718.13it/s]


,image,date,worker,height,width,task,type,region,disease,crop,area,grow,risk,points
0,V006_77_1_18_11_03_13_1_3248b_20201005_57.jpg,2020/10/05,,3024,4032,77,1,4,18,11,3,13,1,"[{'xtl': 908, 'ytl': 916, 'xbr': 3104, 'ybr': ..."
1,V006_77_1_18_11_03_13_1_3248b_20201012_60.jpg,2020/10/12,,3024,4032,77,1,4,18,11,3,13,1,"[{'xtl': 1108, 'ytl': 556, 'xbr': 3356, 'ybr':..."
2,V006_77_1_18_11_03_13_1_3248b_20201012_61.jpg,2020/10/12,,3024,4032,77,1,4,18,11,3,13,1,"[{'xtl': 1152, 'ytl': 772, 'xbr': 3232, 'ybr':..."
3,V006_77_1_18_11_03_13_1_3248b_20201012_64.jpg,2020/10/12,,3024,4032,77,1,4,18,11,3,13,1,"[{'xtl': 540, 'ytl': 544, 'xbr': 2756, 'ybr': ..."
4,V006_77_1_18_11_03_13_1_3248b_20201019_67.jpg,2020/10/19,,3024,4032,77,1,4,18,11,3,13,1,"[{'xtl': 648, 'ytl': 852, 'xbr': 3340, 'ybr': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1962,V006_77_1_19_11_03_12_3_9357q_20201130_28.jpeg,2020/11/30,,3024,4032,77,1,17,19,11,3,12,3,"[{'xtl': 273, 'ytl': 1004, 'xbr': 3056, 'ybr':..."
1963,V006_77_1_19_11_03_12_3_9357q_20201130_29.jpeg,2020/11/30,,3024,4032,77,1,17,19,11,3,12,3,"[{'xtl': 23, 'ytl': 605, 'xbr': 3575, 'ybr': 2..."
1964,V006_77_1_19_11_03_12_3_9357q_20201130_3.jpeg,2020/11/30,,3024,4032,77,1,17,19,11,3,12,3,"[{'xtl': 256, 'ytl': 398, 'xbr': 3576, 'ybr': ..."
1965,V006_77_1_19_11_03_12_3_9357q_20201130_7.jpeg,2020/11/30,,3024,4032,77,1,17,19,11,3,12,3,"[{'xtl': 729, 'ytl': 545, 'xbr': 3735, 'ybr': ..."


In [37]:
disease1_df.to_csv('disease1_df.csv')

#### 질병9

In [40]:
with zipfile.ZipFile(os.path.join(ROOT_DIR, label[2]), 'r') as target_file:
    file_list = target_file.namelist()

In [41]:
disease2_json_data = []

for i in tqdm(range(len(file_list))):
    with zipfile.ZipFile(os.path.join(ROOT_DIR, label[2]), 'r') as zip_ref:
        with zip_ref.open(file_list[i]) as file:
            disease2_json_data.append(json.load(file))

disease2_json_data

100%|██████████████████████████████████████████████████████████████████████████| 19030/19030 [1:02:03<00:00,  5.11it/s]


[{'description': {'image': 'V006_77_1_18_11_03_13_1_3248b_20201005_57_a0000.jpg',
   'date': '2020/10/05',
   'worker': '',
   'height': 3024,
   'width': 4032,
   'task': 77,
   'type': 1,
   'region': 4,
   'original': 'V006_77_1_18_11_03_13_1_3248b_20201005_57.jpg'},
  'annotations': {'disease': 18,
   'crop': 11,
   'area': 3,
   'grow': 13,
   'risk': 1,
   'points': [{'xtl': 908, 'ytl': 712, 'xbr': 3104, 'ybr': 2108}]},
  'augmented': {'brightness': 0.9370387671475437,
   'contrast': 0.8037306995298512,
   'saturation': 0.9335766146012883,
   'hue': 0.0921173427008942,
   'transpose': 0,
   'hflip': 0,
   'vflip': 1}},
 {'description': {'image': 'V006_77_1_18_11_03_13_1_3248b_20201005_57_a0001.jpg',
   'date': '2020/10/05',
   'worker': '',
   'height': 3024,
   'width': 4032,
   'task': 77,
   'type': 1,
   'region': 4,
   'original': 'V006_77_1_18_11_03_13_1_3248b_20201005_57.jpg'},
  'annotations': {'disease': 18,
   'crop': 11,
   'area': 3,
   'grow': 13,
   'risk': 1,
   'p

In [42]:
disease2_df_list = []
for i in tqdm(range(len(disease2_json_data))):
    data = {
    'image': disease2_json_data[i]['description']['image'],
    'date': disease2_json_data[i]['description']['date'],
    'worker': disease2_json_data[i]['description']['worker'],
    'height': disease2_json_data[i]['description']['height'],
    'width': disease2_json_data[i]['description']['width'],
    'task': disease2_json_data[i]['description']['task'],
    'type': disease2_json_data[i]['description']['type'],
    'region': disease2_json_data[i]['description']['region'],
    'disease' : disease2_json_data[i]['annotations']['disease'],
    'crop' : disease2_json_data[i]['annotations']['crop'],
    'area' : disease2_json_data[i]['annotations']['area'],
    'grow' : disease2_json_data[i]['annotations']['grow'],
    'risk' : disease2_json_data[i]['annotations']['risk'],
    'points' : disease2_json_data[i]['annotations']['points']
    }
    disease2_df_list.append(data)
    
disease2_df = pd.DataFrame(disease2_df_list)
disease2_df

100%|█████████████████████████████████████████████████████████████████████████| 19030/19030 [00:00<00:00, 79492.64it/s]


,image,date,worker,height,width,task,type,region,disease,crop,area,grow,risk,points
0,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,,3024,4032,77,1,4,18,11,3,13,1,"[{'xtl': 908, 'ytl': 712, 'xbr': 3104, 'ybr': ..."
1,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,,3024,4032,77,1,4,18,11,3,13,1,"[{'xtl': 927, 'ytl': 916, 'xbr': 3124, 'ybr': ..."
2,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,,4032,3024,77,1,4,18,11,3,13,1,"[{'xtl': 916, 'ytl': 908, 'xbr': 2312, 'ybr': ..."
3,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,,4032,3024,77,1,4,18,11,3,13,1,"[{'xtl': 916, 'ytl': 908, 'xbr': 2312, 'ybr': ..."
4,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,,4032,3024,77,1,4,18,11,3,13,1,"[{'xtl': 916, 'ytl': 927, 'xbr': 2312, 'ybr': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19025,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0005...,2020/11/30,,3024,4032,77,1,17,19,11,3,12,3,"[{'xtl': 644, 'ytl': 438, 'xbr': 3636, 'ybr': ..."
19026,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0006...,2020/11/30,,3024,4032,77,1,17,19,11,3,12,3,"[{'xtl': 644, 'ytl': 497, 'xbr': 3636, 'ybr': ..."
19027,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0007...,2020/11/30,,3024,4032,77,1,17,19,11,3,12,3,"[{'xtl': 644, 'ytl': 497, 'xbr': 3636, 'ybr': ..."
19028,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0008...,2020/11/30,,4032,3024,77,1,17,19,11,3,12,3,"[{'xtl': 497, 'ytl': 396, 'xbr': 2586, 'ybr': ..."


In [43]:
disease2_df.to_csv('disease2_df.csv')

### points에서 좌표값 각각의 열로 만들기

In [34]:
normal_df = pd.read_csv('normal_df.csv', index_col=0)
disease1_df = pd.read_csv('disease1_df.csv', index_col=0)
disease2_df = pd.read_csv('disease2_df.csv', index_col=0)

#### 정상

In [35]:
normal_df.points

0        [{'xtl': 1864, 'ytl': 1516, 'xbr': 2292, 'ybr'...
1        [{'xtl': 1722, 'ytl': 436, 'xbr': 2167, 'ybr':...
2        [{'xtl': 2150, 'ytl': 100, 'xbr': 2727, 'ybr':...
3        [{'xtl': 2147, 'ytl': 102, 'xbr': 2729, 'ybr':...
4        [{'xtl': 1979, 'ytl': 832, 'xbr': 2719, 'ybr':...
                               ...                        
13504    [{'xtl': 3, 'ytl': 737, 'xbr': 4027, 'ybr': 20...
13505    [{'xtl': 0, 'ytl': 905, 'xbr': 4023, 'ybr': 17...
13506    [{'xtl': 0, 'ytl': 953, 'xbr': 4031, 'ybr': 16...
13507    [{'xtl': 0, 'ytl': 1105, 'xbr': 4032, 'ybr': 2...
13508    [{'xtl': 0, 'ytl': 1089, 'xbr': 4032, 'ybr': 2...
Name: points, Length: 13509, dtype: object

In [38]:
normal_df.loc[0].points

"[{'xtl': 1864, 'ytl': 1516, 'xbr': 2292, 'ybr': 1880}]"

In [41]:
normal_df['points'] = [ast.literal_eval(normal_df.loc[i].points) for i in range(len(normal_df))]
normal_df['points']

0        [{'xtl': 1864, 'ytl': 1516, 'xbr': 2292, 'ybr'...
1        [{'xtl': 1722, 'ytl': 436, 'xbr': 2167, 'ybr':...
2        [{'xtl': 2150, 'ytl': 100, 'xbr': 2727, 'ybr':...
3        [{'xtl': 2147, 'ytl': 102, 'xbr': 2729, 'ybr':...
4        [{'xtl': 1979, 'ytl': 832, 'xbr': 2719, 'ybr':...
                               ...                        
13504    [{'xtl': 3, 'ytl': 737, 'xbr': 4027, 'ybr': 20...
13505    [{'xtl': 0, 'ytl': 905, 'xbr': 4023, 'ybr': 17...
13506    [{'xtl': 0, 'ytl': 953, 'xbr': 4031, 'ybr': 16...
13507    [{'xtl': 0, 'ytl': 1105, 'xbr': 4032, 'ybr': 2...
13508    [{'xtl': 0, 'ytl': 1089, 'xbr': 4032, 'ybr': 2...
Name: points, Length: 13509, dtype: object

In [42]:
normal_df.loc[0]['points']

[{'xtl': 1864, 'ytl': 1516, 'xbr': 2292, 'ybr': 1880}]

In [44]:
normal_df.loc[0]['points'][0]['ytl']

1516

In [45]:
normal_df_xtl = [normal_df.loc[i]['points'][0]['xtl'] for i in range(len(normal_df))]
normal_df_ytl = [normal_df.loc[i]['points'][0]['ytl'] for i in range(len(normal_df))]
normal_df_xbr = [normal_df.loc[i]['points'][0]['xbr'] for i in range(len(normal_df))]
normal_df_ybr = [normal_df.loc[i]['points'][0]['ybr'] for i in range(len(normal_df))]

In [48]:
normal_points = pd.DataFrame({'xtl' : normal_df_xtl, 'ytl' : normal_df_ytl, 'xbr' : normal_df_xbr, 'ybr' : normal_df_ybr})
normal_points

,xtl,ytl,xbr,ybr
0,1864,1516,2292,1880
1,1722,436,2167,676
2,2150,100,2727,784
3,2147,102,2729,782
4,1979,832,2719,1462
...,...,...,...,...
13504,3,737,4027,2065
13505,0,905,4023,1745
13506,0,953,4031,1697
13507,0,1105,4032,2113


In [49]:
normal_points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13509 entries, 0 to 13508
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   xtl     13509 non-null  int64
 1   ytl     13509 non-null  int64
 2   xbr     13509 non-null  int64
 3   ybr     13509 non-null  int64
dtypes: int64(4)
memory usage: 422.3 KB


In [50]:
normal = pd.concat([normal_df, normal_points], axis=1)
normal

,image,date,worker,height,width,task,type,region,disease,crop,area,grow,risk,points,xtl,ytl,xbr,ybr
0,V006_77_0_00_11_01_13_0_c01_20201013_0000_S01.jpg,2020/10/13,NaN,3024,4032,77,0,NaN,0,11,1,13,0,"[{'xtl': 1864, 'ytl': 1516, 'xbr': 2292, 'ybr'...",1864,1516,2292,1880
1,V006_77_0_00_11_01_13_0_c01_20201013_0000_S01_...,2020/10/13,NaN,3024,4032,77,0,NaN,0,11,1,13,0,"[{'xtl': 1722, 'ytl': 436, 'xbr': 2167, 'ybr':...",1722,436,2167,676
2,V006_77_0_00_11_01_13_0_c01_20201013_0001_S01.jpg,2020/10/13,NaN,3024,4032,77,0,NaN,0,11,1,13,0,"[{'xtl': 2150, 'ytl': 100, 'xbr': 2727, 'ybr':...",2150,100,2727,784
3,V006_77_0_00_11_01_13_0_c01_20201013_0001_S01_...,2020/10/13,NaN,3024,4032,77,0,NaN,0,11,1,13,0,"[{'xtl': 2147, 'ytl': 102, 'xbr': 2729, 'ybr':...",2147,102,2729,782
4,V006_77_0_00_11_01_13_0_c01_20201013_0002_S01.jpg,2020/10/13,NaN,3024,4032,77,0,NaN,0,11,1,13,0,"[{'xtl': 1979, 'ytl': 832, 'xbr': 2719, 'ybr':...",1979,832,2719,1462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13504,V006_77_0_00_11_05_13_0_c42_20201028_0066_S01_...,2020/10/28,NaN,3024,4032,77,0,NaN,0,11,5,13,0,"[{'xtl': 3, 'ytl': 737, 'xbr': 4027, 'ybr': 20...",3,737,4027,2065
13505,V006_77_0_00_11_05_13_0_c42_20201028_0067_S01.jpg,2020/10/28,NaN,3024,4032,77,0,NaN,0,11,5,13,0,"[{'xtl': 0, 'ytl': 905, 'xbr': 4023, 'ybr': 17...",0,905,4023,1745
13506,V006_77_0_00_11_05_13_0_c42_20201028_0067_S01_...,2020/10/28,NaN,3024,4032,77,0,NaN,0,11,5,13,0,"[{'xtl': 0, 'ytl': 953, 'xbr': 4031, 'ybr': 16...",0,953,4031,1697
13507,V006_77_0_00_11_05_13_0_c42_20201028_0068_S01.jpg,2020/10/28,NaN,3024,4032,77,0,NaN,0,11,5,13,0,"[{'xtl': 0, 'ytl': 1105, 'xbr': 4032, 'ybr': 2...",0,1105,4032,2113


In [51]:
normal.xtl.value_counts()

0       396
1        77
6        50
11       37
21       27
       ... 
33        1
28        1
1761      1
1627      1
2222      1
Name: xtl, Length: 2045, dtype: int64

In [52]:
normal.ytl.value_counts()

0       108
956      29
876      23
1026     22
1176     20
       ... 
1667      1
1866      1
419       1
37        1
83        1
Name: ytl, Length: 2151, dtype: int64

In [53]:
normal.xbr.value_counts()

4032    293
4021     73
4031     68
4026     53
4027     36
       ... 
3664      1
3911      1
3195      1
3039      1
1641      1
Name: xbr, Length: 2645, dtype: int64

In [54]:
normal.ybr.value_counts()

3024    52
2736    20
2432    20
2656    19
2659    18
        ..
376      1
377      1
315      1
465      1
1513     1
Name: ybr, Length: 2658, dtype: int64

#### 질병1

In [59]:
disease1_df['points'] = [ast.literal_eval(disease1_df.loc[i].points) for i in range(len(disease1_df))]

disease1_df_xtl = [disease1_df.loc[i]['points'][0]['xtl'] for i in range(len(disease1_df))]
disease1_df_ytl = [disease1_df.loc[i]['points'][0]['ytl'] for i in range(len(disease1_df))]
disease1_df_xbr = [disease1_df.loc[i]['points'][0]['xbr'] for i in range(len(disease1_df))]
disease1_df_ybr = [disease1_df.loc[i]['points'][0]['ybr'] for i in range(len(disease1_df))]

disease1_points = pd.DataFrame({'xtl' : disease1_df_xtl, 'ytl' : disease1_df_ytl, 'xbr' : disease1_df_xbr, 'ybr' : disease1_df_ybr})
disease1_points

,xtl,ytl,xbr,ybr
0,908,916,3104,2312
1,1108,556,3356,2260
2,1152,772,3232,2420
3,540,544,2756,2224
4,648,852,3340,2524
...,...,...,...,...
1962,273,1004,3056,2244
1963,23,605,3575,2571
1964,256,398,3576,2924
1965,729,545,3735,2612


In [60]:
disease1_points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1967 entries, 0 to 1966
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   xtl     1967 non-null   int64
 1   ytl     1967 non-null   int64
 2   xbr     1967 non-null   int64
 3   ybr     1967 non-null   int64
dtypes: int64(4)
memory usage: 61.6 KB


In [61]:
disease1 = pd.concat([disease1_df, disease1_points], axis=1)
disease1

,image,date,worker,height,width,task,type,region,disease,crop,area,grow,risk,points,xtl,ytl,xbr,ybr
0,V006_77_1_18_11_03_13_1_3248b_20201005_57.jpg,2020/10/05,NaN,3024,4032,77,1,4,18,11,3,13,1,"[{'xtl': 908, 'ytl': 916, 'xbr': 3104, 'ybr': ...",908,916,3104,2312
1,V006_77_1_18_11_03_13_1_3248b_20201012_60.jpg,2020/10/12,NaN,3024,4032,77,1,4,18,11,3,13,1,"[{'xtl': 1108, 'ytl': 556, 'xbr': 3356, 'ybr':...",1108,556,3356,2260
2,V006_77_1_18_11_03_13_1_3248b_20201012_61.jpg,2020/10/12,NaN,3024,4032,77,1,4,18,11,3,13,1,"[{'xtl': 1152, 'ytl': 772, 'xbr': 3232, 'ybr':...",1152,772,3232,2420
3,V006_77_1_18_11_03_13_1_3248b_20201012_64.jpg,2020/10/12,NaN,3024,4032,77,1,4,18,11,3,13,1,"[{'xtl': 540, 'ytl': 544, 'xbr': 2756, 'ybr': ...",540,544,2756,2224
4,V006_77_1_18_11_03_13_1_3248b_20201019_67.jpg,2020/10/19,NaN,3024,4032,77,1,4,18,11,3,13,1,"[{'xtl': 648, 'ytl': 852, 'xbr': 3340, 'ybr': ...",648,852,3340,2524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1962,V006_77_1_19_11_03_12_3_9357q_20201130_28.jpeg,2020/11/30,NaN,3024,4032,77,1,17,19,11,3,12,3,"[{'xtl': 273, 'ytl': 1004, 'xbr': 3056, 'ybr':...",273,1004,3056,2244
1963,V006_77_1_19_11_03_12_3_9357q_20201130_29.jpeg,2020/11/30,NaN,3024,4032,77,1,17,19,11,3,12,3,"[{'xtl': 23, 'ytl': 605, 'xbr': 3575, 'ybr': 2...",23,605,3575,2571
1964,V006_77_1_19_11_03_12_3_9357q_20201130_3.jpeg,2020/11/30,NaN,3024,4032,77,1,17,19,11,3,12,3,"[{'xtl': 256, 'ytl': 398, 'xbr': 3576, 'ybr': ...",256,398,3576,2924
1965,V006_77_1_19_11_03_12_3_9357q_20201130_7.jpeg,2020/11/30,NaN,3024,4032,77,1,17,19,11,3,12,3,"[{'xtl': 729, 'ytl': 545, 'xbr': 3735, 'ybr': ...",729,545,3735,2612


In [65]:
disease1.ybr.value_counts()

3024    24
1908    16
2476     8
2328     8
2316     8
        ..
2363     1
2397     1
1285     1
1773     1
2526     1
Name: ybr, Length: 1123, dtype: int64

#### 질병9

In [66]:
disease2_df['points'] = [ast.literal_eval(disease2_df.loc[i].points) for i in range(len(disease2_df))]

disease2_df_xtl = [disease2_df.loc[i]['points'][0]['xtl'] for i in range(len(disease2_df))]
disease2_df_ytl = [disease2_df.loc[i]['points'][0]['ytl'] for i in range(len(disease2_df))]
disease2_df_xbr = [disease2_df.loc[i]['points'][0]['xbr'] for i in range(len(disease2_df))]
disease2_df_ybr = [disease2_df.loc[i]['points'][0]['ybr'] for i in range(len(disease2_df))]

disease2_points = pd.DataFrame({'xtl' : disease2_df_xtl, 'ytl' : disease2_df_ytl, 'xbr' : disease2_df_xbr, 'ybr' : disease2_df_ybr})
disease2_points

,xtl,ytl,xbr,ybr
0,908,712,3104,2108
1,927,916,3124,2312
2,916,908,2312,3104
3,916,908,2312,3104
4,916,927,2312,3124
...,...,...,...,...
19025,644,438,3636,2526
19026,644,497,3636,2586
19027,644,497,3636,2586
19028,497,396,2586,3388


In [67]:
disease2_points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19030 entries, 0 to 19029
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   xtl     19030 non-null  int64
 1   ytl     19030 non-null  int64
 2   xbr     19030 non-null  int64
 3   ybr     19030 non-null  int64
dtypes: int64(4)
memory usage: 594.8 KB


In [69]:
disease2 = pd.concat([disease2_df, disease2_points], axis=1)
disease2

,image,date,worker,height,width,task,type,region,disease,crop,area,grow,risk,points,xtl,ytl,xbr,ybr
0,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,NaN,3024,4032,77,1,4,18,11,3,13,1,"[{'xtl': 908, 'ytl': 712, 'xbr': 3104, 'ybr': ...",908,712,3104,2108
1,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,NaN,3024,4032,77,1,4,18,11,3,13,1,"[{'xtl': 927, 'ytl': 916, 'xbr': 3124, 'ybr': ...",927,916,3124,2312
2,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,NaN,4032,3024,77,1,4,18,11,3,13,1,"[{'xtl': 916, 'ytl': 908, 'xbr': 2312, 'ybr': ...",916,908,2312,3104
3,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,NaN,4032,3024,77,1,4,18,11,3,13,1,"[{'xtl': 916, 'ytl': 908, 'xbr': 2312, 'ybr': ...",916,908,2312,3104
4,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,NaN,4032,3024,77,1,4,18,11,3,13,1,"[{'xtl': 916, 'ytl': 927, 'xbr': 2312, 'ybr': ...",916,927,2312,3124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19025,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0005...,2020/11/30,NaN,3024,4032,77,1,17,19,11,3,12,3,"[{'xtl': 644, 'ytl': 438, 'xbr': 3636, 'ybr': ...",644,438,3636,2526
19026,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0006...,2020/11/30,NaN,3024,4032,77,1,17,19,11,3,12,3,"[{'xtl': 644, 'ytl': 497, 'xbr': 3636, 'ybr': ...",644,497,3636,2586
19027,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0007...,2020/11/30,NaN,3024,4032,77,1,17,19,11,3,12,3,"[{'xtl': 644, 'ytl': 497, 'xbr': 3636, 'ybr': ...",644,497,3636,2586
19028,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0008...,2020/11/30,NaN,4032,3024,77,1,17,19,11,3,12,3,"[{'xtl': 497, 'ytl': 396, 'xbr': 2586, 'ybr': ...",497,396,2586,3388


In [71]:
disease2.ytl.value_counts()

0       313
1        72
852      57
892      55
736      42
       ... 
2702      1
3504      1
2125      1
758       1
2983      1
Name: ytl, Length: 2107, dtype: int64

In [72]:
disease2.disease.unique()

array([18,  1, 19, 15], dtype=int64)

In [73]:
disease1.disease.unique()

array([18,  1, 19,  3, 15], dtype=int64)

In [74]:
normal.disease.unique()

array([ 0, 11], dtype=int64)

### 하나의 데이터프레임으로 만들기 위해 'file' 열 추가해서 정상, 질병1, 질병9 구분

In [77]:
normal['file'] = 0
disease1['file'] = 1
disease2['file'] = 9

In [80]:
disease2.head(2)

,image,date,worker,height,width,task,type,region,disease,crop,area,grow,risk,points,xtl,ytl,xbr,ybr,file
0,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,NaN,3024,4032,77,1,4,18,11,3,13,1,"[{'xtl': 908, 'ytl': 712, 'xbr': 3104, 'ybr': ...",908,712,3104,2108,9
1,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,NaN,3024,4032,77,1,4,18,11,3,13,1,"[{'xtl': 927, 'ytl': 916, 'xbr': 3124, 'ybr': ...",927,916,3124,2312,9


### 하나의 데이터프레임으로 합치기

In [81]:
df = pd.concat([normal, disease1, disease2])
df

,image,date,worker,height,width,task,type,region,disease,crop,area,grow,risk,points,xtl,ytl,xbr,ybr,file
0,V006_77_0_00_11_01_13_0_c01_20201013_0000_S01.jpg,2020/10/13,NaN,3024,4032,77,0,NaN,0,11,1,13,0,"[{'xtl': 1864, 'ytl': 1516, 'xbr': 2292, 'ybr'...",1864,1516,2292,1880,0
1,V006_77_0_00_11_01_13_0_c01_20201013_0000_S01_...,2020/10/13,NaN,3024,4032,77,0,NaN,0,11,1,13,0,"[{'xtl': 1722, 'ytl': 436, 'xbr': 2167, 'ybr':...",1722,436,2167,676,0
2,V006_77_0_00_11_01_13_0_c01_20201013_0001_S01.jpg,2020/10/13,NaN,3024,4032,77,0,NaN,0,11,1,13,0,"[{'xtl': 2150, 'ytl': 100, 'xbr': 2727, 'ybr':...",2150,100,2727,784,0
3,V006_77_0_00_11_01_13_0_c01_20201013_0001_S01_...,2020/10/13,NaN,3024,4032,77,0,NaN,0,11,1,13,0,"[{'xtl': 2147, 'ytl': 102, 'xbr': 2729, 'ybr':...",2147,102,2729,782,0
4,V006_77_0_00_11_01_13_0_c01_20201013_0002_S01.jpg,2020/10/13,NaN,3024,4032,77,0,NaN,0,11,1,13,0,"[{'xtl': 1979, 'ytl': 832, 'xbr': 2719, 'ybr':...",1979,832,2719,1462,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19025,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0005...,2020/11/30,NaN,3024,4032,77,1,17.0,19,11,3,12,3,"[{'xtl': 644, 'ytl': 438, 'xbr': 3636, 'ybr': ...",644,438,3636,2526,9
19026,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0006...,2020/11/30,NaN,3024,4032,77,1,17.0,19,11,3,12,3,"[{'xtl': 644, 'ytl': 497, 'xbr': 3636, 'ybr': ...",644,497,3636,2586,9
19027,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0007...,2020/11/30,NaN,3024,4032,77,1,17.0,19,11,3,12,3,"[{'xtl': 644, 'ytl': 497, 'xbr': 3636, 'ybr': ...",644,497,3636,2586,9
19028,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0008...,2020/11/30,NaN,4032,3024,77,1,17.0,19,11,3,12,3,"[{'xtl': 497, 'ytl': 396, 'xbr': 2586, 'ybr': ...",497,396,2586,3388,9


In [82]:
df.to_csv('total_df.csv')